# Prerequisites : 

In [1]:
# TensorFlow
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist  # MNIST dataset
from tensorflow.keras.utils import to_categorical

print("TF: ", tf.__version__)

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

print("Torch: ", torch.__version__)

TF:  2.17.0
Torch:  2.4.0+cpu


# TensorFlow

In [2]:
# Train a simple CNN model on MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()  # MNIST data with handwritten digits

# Reshape and normalize the images
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255  # 60000 images, 28x28 pixels, 1 channel (grayscale)
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255    # 10000 images, 28x28 pixels, 1 channel (grayscale)

# One-hot encode the labels (categorical encoding)
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=2, batch_size=64)

print(model.summary())
test_loss, test_acc = model.evaluate(test_images, test_labels)

Epoch 1/2


C:\Users\Lee\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8609 - loss: 0.4314
Epoch 2/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9824 - loss: 0.0576


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 3, 3, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 576)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 279,968 (1.07 MB)

 Trainable params: 93,322 (364.54 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 186,646 (729.09 KB)

None
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9846 - loss: 0.0442


# PyTorch

In [3]:
# Train a simple CNN model on MNIST dataset
transform = transforms.Compose([  
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])  

# Load MNIST data
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)  # 1 input channel, 32 output channels, 3x3 kernel, stride 1
        self.conv2 = nn.Conv2d(32, 64, 3, 1) # 32 input channels, 64 output channels, 3x3 kernel, stride 1
        self.conv3 = nn.Conv2d(64, 64, 3, 1) # 64 input channels, 64 output channels, 3x3 kernel, stride 1
        self.fc1 = nn.Linear(64*3*3, 128)    # 64 input features, 128 output features
        self.fc2 = nn.Linear(128, 10)        # 10 output classes

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)          # Max pooling with 2x2 kernel
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = x.view(-1, 64*3*3)              # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)                     # No activation function
        return torch.log_softmax(x, dim=1)  # Log softmax for output

model = SimpleCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Epoch {epoch + 1}, Batch {batch_idx}, Loss: {loss.item()}')

# Model evaluation

model.eval()
test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
accuracy = 100. * correct / len(test_loader.dataset)
print(f'Test Loss: {test_loss}, Test Accuracy: {accuracy}%')

Epoch 1, Batch 0, Loss: 2.302431106567383
Epoch 1, Batch 100, Loss: 0.15865811705589294
Epoch 1, Batch 200, Loss: 0.13975650072097778
Epoch 1, Batch 300, Loss: 0.08470036834478378
Epoch 1, Batch 400, Loss: 0.09769904613494873
Epoch 1, Batch 500, Loss: 0.034707680344581604
Epoch 1, Batch 600, Loss: 0.10764718800783157
Epoch 1, Batch 700, Loss: 0.018181776627898216
Epoch 1, Batch 800, Loss: 0.08012817054986954
Epoch 1, Batch 900, Loss: 0.008787793107330799
Epoch 2, Batch 0, Loss: 0.015714023262262344
Epoch 2, Batch 100, Loss: 0.07036981731653214
Epoch 2, Batch 200, Loss: 0.04166674613952637
Epoch 2, Batch 300, Loss: 0.053045108914375305
Epoch 2, Batch 400, Loss: 0.022218424826860428
Epoch 2, Batch 500, Loss: 0.0410098135471344
Epoch 2, Batch 600, Loss: 0.056796666234731674
Epoch 2, Batch 700, Loss: 0.02557995915412903
Epoch 2, Batch 800, Loss: 0.07144532352685928
Epoch 2, Batch 900, Loss: 0.09989520162343979
Test Loss: 3.6718601826578376e-05, Test Accuracy: 98.82%


# ---- End ----

In [6]:
# Scalar
scalar = torch.tensor(7)
scalar

scalar.item()

# Vector
vector = torch.tensor([7, 7])
vector

# Matrix
matrix = torch.tensor([[1, 2], 
                       [3, 4]])
matrix

# Tensor
tensor = torch.tensor([[[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]]])
tensor

tensor([[[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]])